In [1]:
from ssgan_exp import make_trainer, make_parser, set_seed

/home/jovyan/work/local/trainers.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import pandas as pd
import torch
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

In [3]:
parser = make_parser()
cmd_args = 'p --dataset mnist --epochs 10 --perc_labeled 0.00167 --lr 0.003 --noise_dist uniform --use_weight_norm --load_ckpt ./e10_ckpt_ssgan_mnist_perclabeled0,00167_noisesize100_noiseuniform_lr0,003_featmatch1_weightnorm1_gfaNone_equnion1_seed1000.pth'
# cmd_args = 'p --dataset mreo --epochs 100 --perc_labeled 0.08 --lr 0.0006 --noise_dist normal --no_eq_union'

args = parser.parse_args(cmd_args.split(' ')[1:])
print(args)
set_seed(args.seed, use_cuda=use_cuda)

Namespace(batch_size=64, dataset='mnist', epochs=10, eq_union=True, feature_matching=True, gen_final_act=None, load_ckpt='./e10_ckpt_ssgan_mnist_perclabeled0,00167_noisesize100_noiseuniform_lr0,003_featmatch1_weightnorm1_gfaNone_equnion1_seed1000.pth', lr=0.003, noise_dist='uniform', noise_size=100, perc_labeled=0.00167, seed=1000, weight_norm=True)


In [4]:
t = make_trainer(args, to_viz=True)

dataset sizes: train 60000, test 10000
perc labeled: 0.00167, num labeled: 100, num unlabeled: 59900, labeled_i: 60000
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 'fake': 9}


In [5]:
s = 'ckpt_ssgan_{}_perclabeled{}_noisesize{}_noise{}_lr{}_featmatch{}_weightnorm{}_gfa{}_equnion{}_seed{}'.format(args.dataset, \
     args.perc_labeled, args.noise_size, args.noise_dist, args.lr,
     int(args.feature_matching), int(args.weight_norm), args.gen_final_act,
     int(args.eq_union), args.seed)
s = s.replace('.', ',')
print('exp', s)

exp ckpt_ssgan_mnist_perclabeled0,00167_noisesize100_noiseuniform_lr0,003_featmatch1_weightnorm1_gfaNone_equnion1_seed1000


In [6]:
if args.load_ckpt: t.load(args.load_ckpt)

In [7]:
from bokeh.io import output_notebook
from bokeh.plotting import show
from bokeh.layouts import gridplot

In [8]:
output_notebook()
grid_D = t.history['D'].viz()
t.history['D'].viz_handle = show(gridplot(grid_D), notebook_handle=True)

Loading BokehJS ...

In [9]:
t.run(max_epoch=args.epochs)

In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):
    print('discriminator history:')
    print(t.history['D'].to_df())
    print('-' * 50)
    print('generator history:')
    print(t.history['G'].to_df())

discriminator history:
    phase  epoch  AverageAccuracy  FakeAccuracy    loss_D  loss_labeled  \
0   train  1      0.941217         0.196250      0.470648  0.162778       
1   test   1      0.773700        NaN            0.000000  0.000000       
2   train  2      0.998900         0.162050      0.306663  0.006643       
3   test   2      0.827300        NaN            0.000000  0.000000       
4   train  3      0.999783         0.244967      0.354957  0.003285       
5   test   3      0.867600        NaN            0.000000  0.000000       
6   train  4      0.999900         0.224933      0.347433  0.002196       
7   test   4      0.875600        NaN            0.000000  0.000000       
8   train  5      0.999867         0.234467      0.327110  0.001584       
9   test   5      0.939400        NaN            0.000000  0.000000       
10  train  6      0.999933         0.327483      0.305535  0.001232       
11  test   6      0.940400        NaN            0.000000  0.000000       
12

In [ ]:
t.save(name=s, path='./')